In [16]:
with open('input.txt', 'r', encoding='utf8') as f:
    text = f.read()
    print("Lengde av datasett i antall karakterer:", len(text))

    #Ser på første 1000 karakterer først
    print(text[:1000])

    #Lag liste av unike karakterer i teksten
    chars = sorted(list(set(text)))
    vocab_size = len(chars) #Antall av hver bokstav
    print(''.join(chars))
    print(vocab_size)

    #Mappe fra bokstaver til indeks og omvendt
    stoi = {ch:i for i, ch in enumerate(chars)} #stoi = string to index
    itos = {i:ch for i, ch in enumerate(chars)} #itos = index to string
    encode = lambda s: [stoi[c] for c in s] #Tekst til liste av indekser
    decode = lambda l: ''.join([itos[i] for i in l]) #Liste

    print('encode: ', encode('Selma kan kooodeee'))
    print('decode: ', decode([24, 27, 24])) #LOL

    #lage tokenized versjon av hele teksten
    import torch
    data = torch.tensor(encode(text), dtype=torch.long)
    print(data.shape, data.dtype)
    print(data[:1000]) #første 1000 bokstaver som indekser

    #Trening og valideringsdeler av datasettet
    n = int(0.9*len(data))
    train_data = data[:n] #alt foran n som er 90%
    val_data = data[n:] #alt etter n som er siste 10%
    print(train_data.shape, val_data.shape)

Lengde av datasett i antall karakterer: 1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this i

In [17]:
block_size = 8
train_data[:block_size + 1]

x = train_data[:block_size] #input
y = train_data[1:block_size + 1] #target
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'Når input er {context.tolist()} er target: {target}')

Når input er [18] er target: 47
Når input er [18, 47] er target: 56
Når input er [18, 47, 56] er target: 57
Når input er [18, 47, 56, 57] er target: 58
Når input er [18, 47, 56, 57, 58] er target: 1
Når input er [18, 47, 56, 57, 58, 1] er target: 15
Når input er [18, 47, 56, 57, 58, 1, 15] er target: 47
Når input er [18, 47, 56, 57, 58, 1, 15, 47] er target: 58


In [18]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y 

xb, yb = get_batch('train')
print('inputs: ')
print(xb.shape)
print(xb)
print('target')
print(yb.shape)
print(yb)

print('---')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'Bokstav {t} i batch {b}, når input er {context.tolist()} er target: {target}')  

inputs: 
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
target
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---
Bokstav 0 i batch 0, når input er [24] er target: 43
Bokstav 1 i batch 0, når input er [24, 43] er target: 58
Bokstav 2 i batch 0, når input er [24, 43, 58] er target: 5
Bokstav 3 i batch 0, når input er [24, 43, 58, 5] er target: 57
Bokstav 4 i batch 0, når input er [24, 43, 58, 5, 57] er target: 1
Bokstav 5 i batch 0, når input er [24, 43, 58, 5, 57, 1] er target: 46
Bokstav 6 i batch 0, når input er [24, 43, 58, 5, 57, 1, 46] er target: 43
Bokstav 7 i batch 0, når input er [24, 43, 58, 5, 57, 1, 46, 43] er target: 39
Bokstav 0 i batch 1, når input er [44] er target: 53
Bokstav 1 i batch 1, når 

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # (B,T,C

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] # (B,C)
            probs = F.softmax(logits, dim=-1) # (B,C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            idx = torch.cat((idx, idx_next), dim=1) # (B,T+1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape) #(B,T,C)
print(loss) #(B*T,)

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [20]:
#create optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [25]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()    

print(loss.item())

2.530308723449707


In [26]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


HAs; n og ARKEve tous oone ntanthintis n Thimed;
Houchy me t ckin by mapa, owis.
Andean she dva
Age f:
Yome as hy rercad locheres.
I oly I t onddocint biss be ceed vemmy I ngowhinothabld
tat hel I omporyor yo t.
IIORIIN s, ir Bewilf sof ate k suser s t usprgr!
TUKiler igerindsolle CESIOFormegine s f pr'my.
US: s lele.
ORe man thethes smm mu hen ffrend t whickitho.
f o n t s lewr ny ay Seerklour, ad, othendete t rea'
LYo w? hor tandinthes s h anofok, y thupe s celd
Ay ayoe imovitod ane'l,
G st yo


In [27]:
#toy example:

torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
print(x.shape) #(4,8,2) 

torch.Size([4, 8, 2])


In [28]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b, t] = torch.mean(x[b, :t+1], 0)

In [ ]:
#version 2 med matriseoperasjoner
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B,T,T) @ (B,T,C) -> (B,T,C)
torch.allclose(xbow, xbow2)

True

In [54]:
#version 4: self-attention
torch.manual_seed(1337)
B,T,C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k=key(x) #(B,T,16)
q=query(x) #(B,T,16)
wei = q @ k.transpose(-2, -1) * head_size**-0.

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x # (B,T,T) @ (B,T,C)

v = value(x) #(B,T,16)
out = wei @ v #(B,T,16)




In [55]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089

In [45]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [44]:
#version 3 med softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x # (B,T,T) @ (B,T,C) -> (B,T,C)
torch.allclose(xbow, xbow3)

True

In [39]:
torch.tril(torch.ones(3, 3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [40]:
torch.manual_seed(42)
a = torch.tril(torch.ones((3,3)))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print ( 'a=')
print(a)
print('---')
print('b=')
print(b)
print('---')
print('c=')
print(c)


a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
---
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
